### 감정분석
* 텍스트에 숨겨져있는 저자의 의도/정보를 찾아내는 방법
* Opinion Mining, Sentiment Mining
* 긍정/부정으로 나눠지는 형태
* text classification

### 텍스트 분류
* 벡터형태로된 텍스트를 분류
* 주로 딥러닝(LSTM) 사용
* 예) QnA, 챗봇(답변), 질문분류...

In [6]:
# 파일읽어오는 함수
def readFile(path):
    with open(path, encoding='utf-8') as f:
        doc = []
        for line in f.read().splitlines()[1:]:
            doc.append(line.split('\t')[1:])
    return doc

In [7]:
# NSMC 학습데이터와 테스트데이터 파일을 읽어오기
train = readFile('ratings_train.txt')
test = readFile('ratings_test.txt')

In [8]:
train[:3]

[['아 더빙.. 진짜 짜증나네요 목소리', '0'],
 ['흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'],
 ['너무재밓었다그래서보는것을추천한다', '0']]

In [9]:
# 전처리 함수
def preprocessing(txt):
    import re
    txt = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', txt)
    sw = set()
    with open('stopwords-ko.txt', encoding='utf-8') as f:
        for word in f:
            sw.add(word.replace('\n',''))
    tokens = []
    from eunjeon import Mecab
    ko = Mecab()
    for t in ko.morphs(txt):
        if t not in sw and len(t) > 1:
            tokens.append(t)
    return tokens
# nsmc텍스트 전처리 함수
def make(data):
    result = []
    for line in data:
        v = preprocessing(line[0])
        if  v:
            result.append((v, line[1])) # 전처리한텍스트와 label을 튜플로 추가
    return result

In [10]:
train_data = make(train)

In [11]:
train_data[:3]

[(['진짜', '짜증', '네요', '목소리'], '0'),
 (['포스터', '보고', '초딩', '영화', '오버', '연기', '가볍', '구나'], '1'),
 (['너무', '밓었', '추천', '한다'], '0')]

In [12]:
test_data = make(test)

In [13]:
test_data[:3]

[(['평점', '나쁘', '짜리', '더더욱', '잖아'], '0'),
 (['지루', '은데', '완전', '막장'], '0'),
 (['어도', '텐데', '나와서', '심기', '불편'], '0')]

In [14]:
import pickle
with open('train_data.pkl', 'wb') as f:
    pickle.dump(train_data, f)
with open('test_data.pkl', 'wb') as f:
    pickle.dump(test_data, f)

In [15]:
len(train_data), len(test_data)

(146124, 48760)

In [5]:
import pickle
with open('train_data.pkl', 'rb') as f:
    train_data=pickle.load(f)
with open('test_data.pkl', 'rb') as f:
    test_data=pickle.load(f)

EOFError: Ran out of input

In [16]:
len(train_data), len(test_data)

(146124, 48760)

### NSMC LSTM 분류 모델링

In [28]:
# 데이터 전처리
# x = ['', '','','','']
# y = ['0','1','0']
import numpy as np
train_x = [d for d, _ in train_data]
test_x = [d for d, _ in test_data]
train_y = np.array([int(l) for _, l in train_data])
test_y = np.array([int(l) for _, l in test_data])

In [29]:
train_x[:3], train_y[:3]

([['진짜', '짜증', '네요', '목소리'],
  ['포스터', '보고', '초딩', '영화', '오버', '연기', '가볍', '구나'],
  ['너무', '밓었', '추천', '한다']],
 array([0, 1, 0]))

In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer # 토크나이저
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [30]:
tk = Tokenizer(num_words=35000) # 최대단어수 제한
tk.fit_on_texts(train_x) # 텐서플로우에서 사용하기 위한 형태로 토크나이징

In [31]:
train_x = tk.texts_to_sequences(train_x)
test_x = tk.texts_to_sequences(test_x)

In [32]:
train_x[0]

[7, 94, 5, 406]

In [33]:
# 패드시퀀스
train_x = pad_sequences(train_x, padding='pre', maxlen=100)
test_x = pad_sequences(test_x, padding='pre', maxlen=100)

In [34]:
# 학습가능한 형태로 변환
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(10000).batch(128) # 배치사이즈 128
test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(128) # 배치사이즈 128

In [36]:
model = Sequential()
# 입력레이어는 Embedding으로 벡터화
model.add(Embedding(input_dim=35000, output_dim=100))
# RNN
model.add(LSTM(128,dropout=0.2))
# 출력레이어(유닛:1, 이진분류)
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(1))

# 설정
model.compile(optimizer='adam', loss='binary_crossentropy', metrics='acc')

# Early Stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1)

# ModelCheckPoint
mc = tf.keras.callbacks.ModelCheckpoint('nsmc_best', monitor='val_loss', mode='min', save_best_only=True)


# 학습
history=model.fit(train_ds, validation_data=test_ds, epochs=20, batch_size=32, callbacks=[es,mc])

Epoch 1/20
1142/1142 [==============================] - ETA: 0s - loss: 0.5124 - acc: 0.7839

INFO:tensorflow:Assets written to: nsmc_best\assets


INFO:tensorflow:Assets written to: nsmc_best\assets


1142/1142 [==============================] - 276s 240ms/step - loss: 0.5124 - acc: 0.7839 - val_loss: 0.5095 - val_acc: 0.8118
Epoch 2/20
1142/1142 [==============================] - ETA: 0s - loss: 0.4468 - acc: 0.8267

INFO:tensorflow:Assets written to: nsmc_best\assets


INFO:tensorflow:Assets written to: nsmc_best\assets


1142/1142 [==============================] - 353s 309ms/step - loss: 0.4468 - acc: 0.8267 - val_loss: 0.5052 - val_acc: 0.8151
Epoch 3/20
1142/1142 [==============================] - 335s 293ms/step - loss: 0.4348 - acc: 0.8264 - val_loss: 0.5510 - val_acc: 0.8117
Epoch 4/20
1142/1142 [==============================] - ETA: 0s - loss: 0.3857 - acc: 0.8521

INFO:tensorflow:Assets written to: nsmc_best\assets


INFO:tensorflow:Assets written to: nsmc_best\assets


1142/1142 [==============================] - 335s 293ms/step - loss: 0.3857 - acc: 0.8521 - val_loss: 0.4945 - val_acc: 0.8122
Epoch 5/20
1142/1142 [==============================] - 323s 283ms/step - loss: 0.4169 - acc: 0.8419 - val_loss: 0.5683 - val_acc: 0.8140
Epoch 6/20
1142/1142 [==============================] - 330s 289ms/step - loss: 0.3829 - acc: 0.8585 - val_loss: 0.6559 - val_acc: 0.8143
Epoch 7/20
1142/1142 [==============================] - 329s 288ms/step - loss: 0.3534 - acc: 0.8692 - val_loss: 0.6451 - val_acc: 0.8146
Epoch 7: early stopping


In [49]:
# 7회 학습후 early stopping
# 파라미터 수정시 최대 92% 가능
# 가장 정확도가 높았던 모델 로딩
bestmodel = tf.keras.models.load_model('nsmc_best')

In [50]:
# 예측함수
# 텍스트 전처리, 시퀀스, 패딩 후 모델 예측값 출력
# 예측값이 특정기준(여기서는 0.5) 이상이면 긍정, 아니면 부정
def pred_func(txt):
    txt = preprocessing(txt)
    txt = tk.texts_to_sequences([txt])
    txt = pad_sequences(txt, padding='pre', maxlen=100)
    pred = bestmodel.predict(txt)
    if pred > 0.5 : 
        return '긍정'
    else :
        return '부정'

In [51]:
pred_func('이 영화는 정말 최고의 영화입니다.')

1/1 [==============================] - 0s 354ms/step


'긍정'

In [52]:
pred_func('이 영화는 정말 최악의 영화입니다.')

1/1 [==============================] - 0s 19ms/step


'부정'